# Ejercicio en clase: Análisis de Componentes Principales

## 1. PCA con `scikit-learn`: Conjunto de datos USArrests

Versión original en R: James, G., Witten, D., Hastie, T., & Tibishirani, R. (2013). An Introduction to Statistical Learning. Springer Texts in Statistics.

Adaptación original a Python por J. Warmenhoven, actualizada por R. Jordan Crouser.


In [0]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

In [0]:
!wget 'https://drive.google.com/uc?export=download&id=1xKSc7x4ZjcmL-x_w2yGRUAVNdHcWgLcD' -O USArrests.csv

En este ejercicio realizamos Análisis de Componentes Principales (PCA) en el conjunto de datos `USArrests`. Para cada uno de los 50 estados de los EE.UU., el conjunto de datos contiene el número de arrestos por cada 100,000 residentes para cada uno de tres crímenes: Asalto (`Assault`), Asesinato (`Murder`) y Violación (`Rape`). También se registra la variable `UrbanPop` (porcentaje de la población en cada estado que vive en las áreas urbanas).

Las filas del conjunto de datos contienen los 50 estados, en orden alfabético.

In [0]:
df = pd.read_csv('USArrests.csv', index_col=0)
df.info()

In [0]:
df.head()

Empecemos por dar una rápida mirada al valor medio de cada columna:

In [0]:
df.describe()

Vemos de inmediato que los datos tienen valores medios **notablemente** diferentes. También podemos examinar la varianza de las cuatro variables:

In [0]:
df.var()

Como podría haberse esperado, las variables tienen también varianzas notablemente diferentes: la variable `UrbanPop` mide el porcentaje de la población en cada estado que vive en un área urbana, número que no es comparable con el número de crímenes cometidos en cada estado por cada 100,000 individuos. Si dejásemos de escalar las variables antes de realizar PCA, la mayoría de los componentes principales que observaríamos estarían sesgados hacia la variable `Assault`, pues ésta tiene ampliamente la mayor media y la mayor varianza.

Así pues, es importante estandarizar las variables con una media de cero y una desviación estándar de 1 antes de realizar PCA. Podemos hacerlo usando la función `scale` de `sklearn`:

In [0]:
from sklearn.preprocessing import scale
X = scale(df)

# La función scale es equivalente a:
# X = (df - np.mean(df, axis=0)) / np.std(df, axis=0)

Visualicemos los primeros 5 estados de `X` en un dataframe:

In [0]:
pd.DataFrame(X, index=df.index, columns=df.columns).head()

Ahora usaremos la función `PCA()` de `sklearn` para calcular los vectores de cargas *(loadings)*, es decir, los coeficientes que cada variable aporta a cada uno de los componentes principales. Una vez ajustado el modelo, los vectores de carga pueden recuperarse con el atributo `components_`:

In [0]:
from sklearn.decomposition import PCA
pca = PCA().fit(X)
pca_loadings = pca.components_

# También podríamos haberlo calculado usando Singular Value Decomposition
# (U, s, Vt) = np.linalg.svd(X)
# pca_loadings = Vt

# Mostramos las cargas de cada componente principal en un DataFrame
pd.DataFrame(pca_loadings, columns=df.columns, index=['PC1', 'PC2', 'PC3', 'PC4']).head()

Vemos que hay cuatro componentes principales. Esto es de esperarse, porque hay en general $min(n-1, p)$ componentes principales informativos en un conjunto de datos de $n$ observaciones y $p$ variables.

La matriz de vectores de cargas es a veces denominada *matriz de rotación*, porque cuando multiplicamos la matriz X por la transpuesta de la matriz de cargas, nos da las coordenadas de los datos en el sistema de coordenadas rotado. Estas coordenadas son los puntajes *(scores)* de los componentes principales.

Usando la función `transform` de `sklearn`, no necesitamos multiplicar explícitamente los datos por los vectores de carga de los componentes principales para obtener los vectores de puntaje de los componentes principales. La función `transform` nos devuelve una matriz de dimensiones 50 x 4 cuyas columnas son los vectores de puntaje de los componentes principales. Es decir, la k-ésima columna corresponde al vector de puntajes del k-mo componente principal.

In [0]:
# Usando sklearn
pca_scores = pca.transform(X)

# Usando Singular Value Decomposition, los puntajes se obtienen multiplicando U * Sigma
# (Primero reconstruimos la matriz Sigma a partir de los valores singulares s)
# Sigma = np.zeros(X.shape, dtype='float64')   
# Sigma[:X.shape[1],:X.shape[1]] = np.diag(s)
# pca_scores = np.dot(U, Sigma)

# O también multiplicando X * V
# V = Vt.T
# pca_scores = np.dot(X, V)

Visualizamos los puntajes para los primeros estados

In [0]:
df_plot = pd.DataFrame(pca_scores, columns=['PC1', 'PC2', 'PC3', 'PC4'], index=df.index)
df_plot.head()

Como podemos verificar, es posible recuperar por completo el conjunto de datos inicial multiplicando la matriz de puntajes $(U \Sigma)$ por la matriz de cargas $(V^\top)$.

In [0]:
X_rec = np.dot(pca_scores, pca_loadings)
pd.DataFrame(X_rec, index=df.index, columns=df.columns).head()

Definamos una función para construir un **biplot** de los dos primeros componentes principales.

In [0]:
def biplot(loadings, scores, index, columns):
    fig , ax1 = plt.subplots(figsize=(9,7))

    ax1.set_xlim(-3.5,3.5)
    ax1.set_ylim(-3.5,3.5)

    # Nombre de cada estado ubicado por puntajes para cada componente principal
    for i, index in enumerate(index):
        ax1.annotate(index, (scores[i, 0], scores[i, 1]), ha='center', color='blue')

    # Líneas de referencia
    ax1.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
    ax1.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')

    ax1.set_xlabel('Primer Componente Principal')
    ax1.set_ylabel('Segundo Componente Principal')

    # Diagramar los vectores de cargas, superponiendo un segundo eje x, y
    ax2 = ax1.twinx().twiny() 

    ax2.set_ylim(-1,1)
    ax2.set_xlim(-1,1)
    ax2.set_xlabel('Vectores de cargas de los componentes principales', color='red')

    # Vectores de carga
    # La variable 'a' es un pequeño offset para separar las etiquetas de las flechas
    a = 1.07  
    for i, column in enumerate(columns):
        ax2.annotate(column, (loadings[0,i]*a, loadings[1,i]*a), color='red')
        ax2.arrow(0, 0, loadings[0,i], loadings[1,i], color='red')
    plt.show()

In [0]:
biplot(loadings=pca_loadings, scores=pca_scores, index=df.index, columns=df.columns)

La función `PCA()` también nos brinda la varianza explicada por cada componente principal. Podemos acceder a estos valores como sigue:

In [0]:
explained_variance = pca.explained_variance_

# O, usando SVD:
# explained_variance = (s * s) / X.shape[0]

print('Varianza explicada por cada componente principal:')
print(explained_variance)

También podemos obtener la proporción de varianza explicada por cada componente principal:

In [0]:
PVE = pca.explained_variance_ratio_

# O, lo que es lo mismo:
# PVE = explained_variance / np.sum(explained_variance)

print('Proporción de varianza explicada (PVE) por cada componente principal:')
print(PVE)

Podemos ver que el primer componente principal explica 62.0% de la varianza en los datos, el siguiente componente principal explica el 24.7% de la varianza, y así en adelante. Podemos diagramar la proporción de varianza explicada (PVE) por cada componente, como sigue:

In [0]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4], pca.explained_variance_ratio_, '-o')
plt.ylabel('Proporción de Varianza Explicada')
plt.xlabel('Componente Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4])
plt.show()

También podemos usar la función `numpy.cumsum()`, que calcula la suma acumulada de los elementos de un vector numérico, para diagramar la PVE acumulada:

In [0]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4], np.cumsum(pca.explained_variance_ratio_), '-s')
plt.ylabel('Proporción Acumulada de Varianza Explicada')
plt.xlabel('Componente Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4])
plt.show()

# 2. Reducción de dimensionalidad con PCA

(Traducido y adaptado del curso "Machine Learning" de Andrew NG)

En este ejercicio, usarás PCA para realizar reducción de dimensionalidad. Primero experimentarás con un conjunto de datos sencillo en 2D para afianzar la intuición de cómo funciona esta reducción de dimensionalidad, y luego usarás PCA en un conjunto de datos más grande que contiene 5000 rostros.

## 2.1. Conjunto de datos 2D

Empezaremos con un conjunto de datos en 2D que tiene mucha varianza en una dirección y menor variación en la otra. A continuación cargaremos y visualizaremos el conjunto de datos. En esta parte del ejercicio visualizarás lo que ocurre cuando reduces los datos de 2D a 1D. En la práctica, querrás reducir datos de 256 a 50 dimensiones, por decir algo; pero usar datos en menores dimensiones, como en este ejemplo, nos permite visualizar mejor los algoritmos.

In [0]:
!wget 'https://drive.google.com/uc?export=download&id=1Up7hNMhMlzBdDxUJR7l0YbRSM6vB4EYA' -O data1.mat

In [0]:
from scipy.io import loadmat
ex7data1 = loadmat('data1.mat')
X = np.array(ex7data1['X'])
plt.axes().set_aspect('equal', 'datalim')
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X[:,0], X[:,1], marker='o', cmap='prism')
plt.show()

Como hemos visto, es importante normalizar primero los datos sustrayendo el valor medio de cada característica del conjunto de datos y escalando cada dimensión para que todas estén en el mismo rango. Es lo que hacemos a continuación:

In [0]:
from sklearn.preprocessing import scale 

X_norm = scale(X)

Veamos cómo los datos se encuentran ahora normalizados:

In [0]:
print('Antes de normalizar:\n', X[:5])
print('\nDespués de normalizar:\n', X_norm[:5])

Una vez que los datos están normalizados, puedes ejecutar PCA para calcular los componentes principales. 

In [0]:
pca2 = PCA().fit(X_norm)
pca2_loadings = pca2.components_

print('El vector de cargas del primer componente principal es:')
print(pca2_loadings[0,:])

Grafiquemos la dirección de los dos primeros componentes principales:

In [0]:
plt.axes().set_aspect('equal', 'datalim')
plt.scatter(X_norm[:,0], X_norm[:,1], marker='o', cmap='prism')
plt.xlabel('x1')
plt.ylabel('x2')
a = 1.2
plt.annotate('PCA1', (pca2_loadings[0,0]*pca2.explained_variance_[0]*a, pca2_loadings[0,1]*pca2.explained_variance_[0]*a), color='red', ha='right')
plt.arrow(0,0, pca2_loadings[0,0]*pca2.explained_variance_[0], pca2_loadings[0,1]*pca2.explained_variance_[0], color='r', head_width=0.1, length_includes_head=True)
plt.annotate('PCA2', (pca2_loadings[1,0]*pca2.explained_variance_[1]*a, pca2_loadings[1,1]*pca2.explained_variance_[1]*a), color='red', ha='right')
plt.arrow(0,0, pca2_loadings[1,0]*pca2.explained_variance_[1], pca2_loadings[1,1]*pca2.explained_variance_[1], color='r', head_width=0.1, length_includes_head=False)
plt.show()

## 2.2. Proyección de los datos sobre los componentes principales

Después de calcular los componentes principales, puedes usarlos para reducir la dimensión de características de tu conjunto de datos proyectando cada ejemplo sobre un espacio de menores dimensiones, $x^{(i)} \rightarrow z^{(i)}$ (p.ej., proyectando los datos de 2D a 1D).

En la práctica, si estuvieses usando un algoritmo de aprendizaje tal como regresión lineal o quizás redes neuronales, podrías usar los datos proyectados en lugar de los datos originales. Usando los datos proyectados, puedes entrenar tu modelo más rápido pues hay menos dimensiones en los datos de entrada.

Para proyectar los datos sólo necesitas truncar las primeras `K` columnas de la matriz de puntajes.

In [0]:
def projectData(pcaModel, X, K):
    Z = pcaModel.transform(X)[:,:K]
    return Z

In [0]:
# Proyectamos los datos
Z = projectData(pca2, X_norm, 1)

print('La proyección del primer ejemplo sobre la primera dimensión es: %0.5f' % Z[0,0])

## 2.3. Reconstrucción de una aproximación de los datos

Luego de proyectar los datos en un espacio de menores dimensiones, puedes recuperar aproximadamente los datos proyectándolos de regreso en el espacio original de alta dimensión. Para hacerlo, multiplicamos Z por las primeras `K` filas de la matriz de carga.

In [0]:
def recoverData(pcaModel, Z):
    K = Z.shape[1]
    X_rec = np.dot(Z, pcaModel.components_[:K,:])
    return X_rec

In [0]:
X_rec = recoverData(pca2, Z)

## 2.4. Visualización de las proyecciones

Una vez realizada la proyección y reconstrucción aproximada de los datos, veamos en el siguiente diagrama cómo la proyección afecta los datos. Los puntos de datos originales están indicados con color azul, mientras que los puntos de datos proyectados están indicados con color rojo. La proyección retiene efectivamente la información en la dirección del primer componente principal.

In [0]:
plt.axes().set_aspect('equal', 'datalim')
for i in range(X_norm.shape[0]):
    plt.arrow(X_norm[i,0], X_norm[i,1], X_rec[i,0]-X_norm[i,0], X_rec[i,1]-X_norm[i,1], color='k')
plt.scatter(X_norm[:,0], X_norm[:,1], marker='o', c='blue', label='Datos originales')  
plt.scatter(X_rec[:,0], X_rec[:,1], marker='o', c='red', label='Datos recuperados')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend(loc='upper left')
plt.show()

## 2.5. Conjunto de datos de imágenes de rostros

En esta parte del ejercicio, aplicarás PCA a imágenes de rostros para ver cómo puede se puede usar PCA en la práctica para reducción de la dimensionalidad. El archivo `faces.mat` contiene un conjunto de datos `X` con imagénes de 32 x 32 pixeles en tonos de grises. Cada fila corresponde a una imagen (un vector fila de longitud 1024). En el siguiente paso cargaremos `X` y visualizaremos las primeras 100 imágenes.

In [0]:
!wget 'https://drive.google.com/uc?export=download&id=1Bxtbq_L3RCGWDT0cQCQe_7TZdUYY5Am_' -O faces.mat

In [0]:
from scipy.io import loadmat

ex7faces = loadmat('faces.mat')
X = np.array(ex7faces['X'])

In [0]:
from matplotlib.pyplot import figure, imshow, axis

def mostrarImagenes(X, num_cols, shape, title='asdf'):
    num_imagenes = X.shape[0]
    num_filas = np.ceil(num_imagenes / num_cols)
    fig = figure()
    fig.set_size_inches(10, 10)

    for i in range(num_imagenes):
        a=fig.add_subplot(num_filas, num_cols, i +1)
        image = X[i,:].reshape(shape).T
        imshow(image,cmap='Greys_r')
        axis('off')

In [0]:
print('Imágenes originales')
mostrarImagenes(X[:100], 10, [32, 32])

### 2.5.1. PCA en rostros 

Antes de ejecutar PCA en el conjunto de datos, normaliza el conjunto de datos usando la función `scale`. Luego ajusta un modelo PCA a los datos. Observa que cada fila de la matriz de cargas es un vector de longitud $n$ (para el dataset, $n = 1024$). Resulta que podemos visualizar estos componentes principales transformando cada uno de ellos en una matriz 32 x 32 que corresponde a los pixeles del conjunto de datos original. Una vez que hayas ajustado el modelo PCA, mostraremos los primeros 36 componentes principales que describen la mayor varianza. Si deseas, modifica el código para mostrar más componentes principales y ver cómo capturan cada vez más detalles.

In [0]:
from sklearn.preprocessing import scale 

X_norm = scale(X)
pca_faces = PCA().fit(X_norm)
pca_faces_loadings = pca_faces.components_

In [0]:
print('Carga de los primeros 36 componentes principales')
mostrarImagenes(pca_faces_loadings[:36,:], num_cols=6, shape=[32, 32])

### Reducción de dimensionalidad

Ahora que has calculado los principales componentes del conjunto de datos de rostros, puedes usarlos para reducir su dimensionalidad. Esto te permite entrenar tu algoritmo de aprendizaje con un tamaño de entradas menor (p.ej., 100 dimensiones) en lugar de las 1024 dimensiones originales. Esto puede ayudar a acelerar tu algoritmo de aprendizaje.

En la siguiente parte proyectarás el conjunto de datos sobre solamente los primeros 100 componentes principales. Específicamente, cada imagen quedará descrita por un vector $z^{(i)} \in \mathbb{R}^{100}$.

Para entender lo que se pierde en la reducción de dimensionalidad, recuperarás las imágenes a partir de los datos proyectados.

Una vez que hayas calculado las proyecciones y la recuperación de los datos, visualizaremos las imágenes reconstruidas. En la reconstrucción podrás observar que se conserva la estructura general y apariencia de los rostros, y se pierde los detalles más finos. Esta es una notable reducción (más de 10x) en el tamaño del conjunto de datos, que puede ayudar a acelerar significativamente tu algoritmo de aprendizaje. Por ejemplo, si estuvieses entrenando una red neuronal para realizar reconocimiento de personas (dada la imagen de un rostro, identificar a la persona), puedes usar la entrada reducida a 100 dimensiones en lugar de los pixeles originales.

In [0]:
# Proyecta los datos sobre los primeros 100 componentes principales
Z = projectData(pca_faces, X_norm, 100)

# Recupera los datos a partir de la proyección
X_rec = recoverData(pca_faces, Z)

In [0]:
print('Imágenes reconstruidas')
mostrarImagenes(X_rec[:100], 10, [32, 32])

¿Cuál es la proporción acumulada de varianza explicada por los primeros 100 componentes principales del conjunto de datos de rostros? (Redondear a 4 decimales)

In [0]:
print('PVE acumulada de los primeros 100 componentes principales: %.4f' % np.cumsum(pca_faces.explained_variance_ratio_)[99] )

¿Puedes graficar la proporción acumulada de varianza explicada por PCA para este conjunto de datos?

In [0]:
plt.figure(figsize=(7,5))
plt.plot(np.linspace(1,1024,1024), np.cumsum(pca_faces.explained_variance_ratio_))
plt.ylabel('Proporción Acumulada de Varianza Explicada')
plt.xlabel('Componente Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks(np.linspace(100,1100,11))
plt.show()